### Autores: **Angel David Piñeros Sierra**, **Camilo Andrés Roncancio Toca**, **Kelly Johana Solano Calderón**



---



## **Tarea 3 - Semantic Segmentation**





**Sobre las redes convolucionales (CNN):**

Las redes convolucionales son ampliamente utilizadas para el procesamiento de imágenes, teniendo en cuenta sus propiedades y dimensiones. Este tipo de redes esta conformado por capas convolucionales totalmente conectadas que procesan regiones locales de las imágenes de forma independiente, utilizando parámetros que se comparten a lo largo de toda la imagen.

Se tienen en cuenta las siguientes propiedades y procesos:

*   La representación de una imagen se mantiene, así se le hayan aplicado transformaciones (ej: traslación).
*   El proceso de convolución se refiere a la transformación de un vector de entrada $x$ a un vector de salida $z$, donde cada elemento $z_i$ es la suman ponderada de las entradas cercanas a la posición $i$. Los mismos valores de los pesos se utilizan en cada posición del recorrigo, llamandose



**Sobre las redes residuales**


**La arquitectura U-Net**




### **Tratamiento de datos**


In [1]:
'''
Importación de datos
'''

!pip install gdown
id_gdrive = "1gCpZHm9dS2ELSIpS3mJb2RL6PCBk7muF"
nombre_archivo = "coffee_leaves.7z"
!gdown --id $id_gdrive --output $nombre_archivo

/home/angel/Bureau/Tarea 3 IA/.venv/lib/python3.12/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1gCpZHm9dS2ELSIpS3mJb2RL6PCBk7muF
To: /home/angel/Bureau/Tarea 3 IA/coffee_leaves.7z
100%|██████████████████████████████████████| 58.7M/58.7M [00:02<00:00, 20.7MB/s]


In [2]:
%pip install gdown py7zr pillow torch torchvision


  Using cached torch-2.9.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cufft_cu12-11.3.3.83

In [3]:
import py7zr
import os

def descomprimir_archivo(archivo):
    if os.path.exists(archivo[:-3]):
        return
    with py7zr.SevenZipFile(archivo, mode='r') as z:
         z.extractall(path=archivo[:-3])

descomprimir_archivo('coffee_leaves.7z')

In [4]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms


In [5]:
# Normalización y estandarización de las imágenes de ENTRADA usando estadísticas del propio dataset
# 1) ToTensor() normaliza los píxeles de [0, 255] a [0, 1]
# 2) Calculamos media y desviación estándar por canal SOBRE esas imágenes normalizadas
# 3) Usamos esos valores en transforms.Normalize para estandarizar: (x - mean) / std

import numpy as np

train_imgs_dir = os.path.join("coffee_leaves", "coffee_leaves", "train", "imgs")
img_filenames = [f for f in os.listdir(train_imgs_dir) if f.lower().endswith((".png", ".jpg", ".jpeg"))]

means = []
stds = []

for fname in img_filenames:
    img_path = os.path.join(train_imgs_dir, fname)
    img = Image.open(img_path).convert("RGB")
    # 1) Normalización: [0, 255] -> [0, 1]
    img_tensor = transforms.ToTensor()(img)
    # media y desviación estándar POR CANAL
    means.append(img_tensor.mean(dim=(1, 2)))
    stds.append(img_tensor.std(dim=(1, 2)))

means = torch.stack(means, dim=0)
stds = torch.stack(stds, dim=0)

# 2) Estadísticos finales del dataset (promedio sobre todas las imágenes)
IMG_MEAN = means.mean(dim=0)
IMG_STD = stds.mean(dim=0)

print("Media por canal (dataset):", IMG_MEAN)
print("Desviación estándar por canal (dataset):", IMG_STD)

# 3) Transformaciones de entrada para entrenamiento y validación
train_img_transforms = transforms.Compose([
    transforms.ToTensor(),  # normaliza a [0, 1]
    transforms.Normalize(mean=IMG_MEAN.tolist(), std=IMG_STD.tolist()),  # estandariza
])

val_img_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=IMG_MEAN.tolist(), std=IMG_STD.tolist()),
])


Media por canal (dataset): tensor([0.6598, 0.6618, 0.5548])
Desviación estándar por canal (dataset): tensor([0.2315, 0.1936, 0.3129])
